In [11]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl
from dgl.data import register_data_args
from gcn2 import GCN
from ray import tune


In [26]:
(g,), _  = dgl.load_graphs("graph.dgl")
g = g.int().to(0)
features = g.ndata['feat']
labels = g.ndata['label']
train_mask = g.ndata['train_mask'].bool()
val_mask = g.ndata['val_mask'].bool()
test_mask = g.ndata['test_mask'].bool()
in_feats = features.shape[1]
print('in_feats',in_feats)
n_classes = len(torch.unique(labels))    
# add self loop
g = dgl.remove_self_loop(g)
g = dgl.add_self_loop(g)

class arguments():
    def __init__(self):
        self.lr = 0.0008
        self.epochs = 800

def evaluate(model, g, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

def main(config):
    """
    # data from dgl
    (g,), _  = dgl.load_graphs("graph.dgl")
    g = g.int().to(0)
    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask'].bool()
    val_mask = g.ndata['val_mask'].bool()
    test_mask = g.ndata['test_mask'].bool()
    in_feats = features.shape[1]
    print('in_feats',in_feats)
    n_classes = len(torch.unique(labels))    

    # add self loop
    g = dgl.remove_self_loop(g)
    g = dgl.add_self_loop(g)
    """
    # args = arguments()
    # create GCN model
    model = GCN(in_feats, n_classes, config["l1"], config["l2"], config["l3"])
    model.cuda()
    loss_fcn = torch.nn.CrossEntropyLoss()
    # use optimizer
    optimizer = torch.optim.Adam(model.parameters(),
                                lr = config["lr"],
                                weight_decay = config["weight_decay"])


    # initialize graph
    for epoch in range(config["epochs"]):
        model.train()
        # forward
        logits = model(g, features)
        loss = loss_fcn(logits[train_mask], labels[train_mask])

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = evaluate(model, g, features, labels, val_mask)
        #print("Epoch {:05d} | Loss {:.4f} | Accuracy {:.4f} | ". format(epoch, loss.item(), acc))

    acc = evaluate(model, g, features, labels, test_mask)
    #print("Test accuracy {:.2%}".format(acc))
    tune.report(accuracy = acc)


    

in_feats 767


In [33]:
if __name__ == '__main__':
    parameters = {
        #"l1": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
        "l1": tune.choice([128]),
        #"l2": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
        "l2": tune.choice([64]),
        #"l3": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
        "l3": tune.choice([16]),
        #"lr": tune.loguniform(1e-4, 1e-1),
        "lr": tune.sample_from(lambda _: 0.0002*np.random.randint(3, 10)),
        "epochs": tune.sample_from(lambda _: 100*np.random.randint(7, 9)),
        "weight_decay": tune.sample_from(lambda _: 0.0003*np.random.randint(1, 4)),
    }
    #tune.run(main, config = parameters)
    analysis = tune.run(main, config = parameters, num_samples = 50,resources_per_trial={'gpu': 1})    

2021-05-07 17:34:34,956	WARNING worker.py:1115 -- Warning: The actor ImplicitFunc has size 46559969 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.


Trial name,status,loc,epochs,l1,l2,l3,lr,weight_decay
main_780bd_00000,RUNNING,,700,128,64,16,0.0008,0.0009
main_780bd_00001,PENDING,,700,128,64,16,0.001,0.0006
main_780bd_00002,PENDING,,700,128,64,16,0.0016,0.0009
main_780bd_00003,PENDING,,800,128,64,16,0.0012,0.0006
main_780bd_00004,PENDING,,700,128,64,16,0.0006,0.0003
main_780bd_00005,PENDING,,800,128,64,16,0.0018,0.0006
main_780bd_00006,PENDING,,700,128,64,16,0.0016,0.0006
main_780bd_00007,PENDING,,800,128,64,16,0.0012,0.0006
main_780bd_00008,PENDING,,800,128,64,16,0.0014,0.0009
main_780bd_00009,PENDING,,700,128,64,16,0.0018,0.0006


(pid=9402) Using backend: pytorch


(pid=9402) l1 128 l2 64 l3 16


(pid=9426) Using backend: pytorch


...

(pid=10959) l1 128 l2 64 l3 16
Result for main_780bd_00048:
  accuracy: 0.8625454545454545
  date: 2021-05-07_17-42-19
  done: false
  experiment_id: a75fdbab53ad4edbb78fcba7aae494cf
  hostname: 4ce982eb56f7
  iterations_since_restore: 1
  node_ip: 172.17.0.11
  pid: 10926
  time_since_restore: 14.965997219085693
  time_this_iter_s: 14.965997219085693
  time_total_s: 14.965997219085693
  timestamp: 1620409339
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 780bd_00048
  
Result for main_780bd_00048:
  accuracy: 0.8625454545454545
  date: 2021-05-07_17-42-19
  done: true
  experiment_id: a75fdbab53ad4edbb78fcba7aae494cf
  experiment_tag: 48_epochs=800,l1=128,l2=64,l3=16,lr=0.0008,weight_decay=0.0009
  hostname: 4ce982eb56f7
  iterations_since_restore: 1
  node_ip: 172.17.0.11
  pid: 10926
  time_since_restore: 14.965997219085693
  time_this_iter_s: 14.965997219085693
  time_total_s: 14.965997219085693
  timestamp: 1620409339
  timesteps_since_restore: 0
  training_itera

Trial name,status,loc,epochs,l1,l2,l3,lr,weight_decay,iter,total time (s),accuracy
main_780bd_00049,RUNNING,172.17.0.11:10959,800,128,64,16,0.0006,0.0003,1,14.5503,0.872727
main_780bd_00000,TERMINATED,,700,128,64,16,0.0008,0.0009,1,13.6591,0.877091
main_780bd_00001,TERMINATED,,700,128,64,16,0.001,0.0006,1,13.6453,0.883636
main_780bd_00002,TERMINATED,,700,128,64,16,0.0016,0.0009,1,12.996,0.879273
main_780bd_00003,TERMINATED,,800,128,64,16,0.0012,0.0006,1,14.9518,0.883636
main_780bd_00004,TERMINATED,,700,128,64,16,0.0006,0.0003,1,13.1162,0.876364
main_780bd_00005,TERMINATED,,800,128,64,16,0.0018,0.0006,1,14.9799,0.869818
main_780bd_00006,TERMINATED,,700,128,64,16,0.0016,0.0006,1,13.208,0.872727
main_780bd_00007,TERMINATED,,800,128,64,16,0.0012,0.0006,1,15.0515,0.864727
main_780bd_00008,TERMINATED,,800,128,64,16,0.0014,0.0009,1,14.9417,0.874909


Result for main_780bd_00049:
  accuracy: 0.8727272727272727
  date: 2021-05-07_17-42-33
  done: true
  experiment_id: ac9879b88c264685ab7cee5b07430edc
  experiment_tag: 49_epochs=800,l1=128,l2=64,l3=16,lr=0.0006,weight_decay=0.0003
  hostname: 4ce982eb56f7
  iterations_since_restore: 1
  node_ip: 172.17.0.11
  pid: 10959
  time_since_restore: 14.550278425216675
  time_this_iter_s: 14.550278425216675
  time_total_s: 14.550278425216675
  timestamp: 1620409353
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 780bd_00049
  


Trial name,status,loc,epochs,l1,l2,l3,lr,weight_decay,iter,total time (s),accuracy
main_780bd_00000,TERMINATED,,700,128,64,16,0.0008,0.0009,1,13.6591,0.877091
main_780bd_00001,TERMINATED,,700,128,64,16,0.001,0.0006,1,13.6453,0.883636
main_780bd_00002,TERMINATED,,700,128,64,16,0.0016,0.0009,1,12.996,0.879273
main_780bd_00003,TERMINATED,,800,128,64,16,0.0012,0.0006,1,14.9518,0.883636
main_780bd_00004,TERMINATED,,700,128,64,16,0.0006,0.0003,1,13.1162,0.876364
main_780bd_00005,TERMINATED,,800,128,64,16,0.0018,0.0006,1,14.9799,0.869818
main_780bd_00006,TERMINATED,,700,128,64,16,0.0016,0.0006,1,13.208,0.872727
main_780bd_00007,TERMINATED,,800,128,64,16,0.0012,0.0006,1,15.0515,0.864727
main_780bd_00008,TERMINATED,,800,128,64,16,0.0014,0.0009,1,14.9417,0.874909
main_780bd_00009,TERMINATED,,700,128,64,16,0.0018,0.0006,1,13.1547,0.873455


2021-05-07 17:42:33,239	INFO tune.py:549 -- Total run time: 487.41 seconds (487.00 seconds for the tuning loop).


In [32]:
print("Best config: ", analysis.get_best_config(metric="accuracy", mode="max"))
best_trial = analysis.get_best_trial("accuracy", "max")
print("Best trial final validation accuracy: {}".format(best_trial.last_result["accuracy"]))

Best config:  {'l1': 128, 'l2': 64, 'l3': 16, 'lr': 0.0012000000000000001, 'epochs': 700, 'weight_decay': 0.0003}
Best trial final validation accuracy: 0.8872727272727273


In [35]:
print("Best config: ", analysis.get_best_config(metric="accuracy", mode="max"))
best_trial = analysis.get_best_trial("accuracy", "max")
print("Best trial final validation accuracy: {}".format(best_trial.last_result["accuracy"]))

Best config:  {'l1': 128, 'l2': 64, 'l3': 16, 'lr': 0.0018000000000000002, 'epochs': 700, 'weight_decay': 0.0003}
Best trial final validation accuracy: 0.8872727272727273


In [49]:
print("Best trail: ", analysis.get_best_trial(metric="accuracy", mode="max"))
print("Best config: ", analysis.get_best_config(metric="accuracy", mode="max"))
best_trial = analysis.get_best_trial("accuracy", "max")
print("Best trial final validation accuracy: {}".format(best_trial.last_result["accuracy"]))

Best config:  {'l1': 128, 'l2': 64, 'l3': 16, 'lr': 0.0018000000000000002, 'epochs': 700, 'weight_decay': 0.0003}
Best trail:  main_780bd_00028
Best trial final validation accuracy: 0.8872727272727273


In [44]:
dc = analysis.get_all_configs()
for keys in dc:
    print(keys)
    

/root/ray_results/main_2021-05-07_17-34-25/main_780bd_00000_0_epochs=700,l1=128,l2=64,l3=16,lr=0.0008,weight_decay=0.0009_2021-05-07_17-34-34
/root/ray_results/main_2021-05-07_17-34-25/main_780bd_00001_1_epochs=700,l1=128,l2=64,l3=16,lr=0.001,weight_decay=0.0006_2021-05-07_17-34-35
/root/ray_results/main_2021-05-07_17-34-25/main_780bd_00002_2_epochs=700,l1=128,l2=64,l3=16,lr=0.0016,weight_decay=0.0009_2021-05-07_17-34-35
/root/ray_results/main_2021-05-07_17-34-25/main_780bd_00003_3_epochs=800,l1=128,l2=64,l3=16,lr=0.0012,weight_decay=0.0006_2021-05-07_17-34-51
/root/ray_results/main_2021-05-07_17-34-25/main_780bd_00004_4_epochs=700,l1=128,l2=64,l3=16,lr=0.0006,weight_decay=0.0003_2021-05-07_17-34-52
/root/ray_results/main_2021-05-07_17-34-25/main_780bd_00005_5_epochs=800,l1=128,l2=64,l3=16,lr=0.0018,weight_decay=0.0006_2021-05-07_17-35-08
/root/ray_results/main_2021-05-07_17-34-25/main_780bd_00006_6_epochs=700,l1=128,l2=64,l3=16,lr=0.0016,weight_decay=0.0006_2021-05-07_17-35-10
/root/r